## **RQ3**

> _Em modelos globais para previsão multi-horizonte de poluentes, quando variáveis exógenas (meteorologia + co-poluentes) geram ganho real de desempenho — e esse ganho é explicável por atribuições consistentes e fisicamente plausíveis?_

* Avaliação empírica (não arquitetural) do impacto de exógenas;
* Horizontes de 7, 14 e 30 dias;
* Análise explícita de:
   * quando exógenas ajudam,
   * quando não ajudam,
   * possíveis razões físicas e estatísticas.

## Libraries

In [2]:
import os
os.environ['NIXTLA_ID_AS_COL'] = '1'

import optuna
import itertools
import shutil
import time
import functools
import gc
import requests

import pandas as pd
import numpy as np
np.random.seed(1)

import plotly.graph_objects as go
import plotly.express as px
import plotly.subplots
import plotly.io as pio
from graphmodex import plotlymodex
pio.renderers.default = 'notebook'

import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib as mpl

import joblib
import pickle
from IPython.display import clear_output

c:\Users\gustavo.filho\Documents\Python\Masters_New\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import neuralforecast
import mlforecast
import statsforecast
import utilsforecast
import coreforecast

from statsforecast import StatsForecast
from statsforecast.models import (
    Naive, SeasonalNaive,
    AutoARIMA, AutoCES, AutoETS, AutoTheta,
)

from mlforecast import MLForecast
import lightgbm as lgb
from sklearn.linear_model import LinearRegression
from mlforecast.lag_transforms import ExpandingMean, RollingMean
from mlforecast.target_transforms import Differences
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor

from neuralforecast import NeuralForecast
from neuralforecast.models import (
    NBEATS, NHITS, NBEATSx,
    GRU, Informer, LSTM
)
from neuralforecast.losses.pytorch import MSE, SMAPE, MAE

from mlforecast.utils import PredictionIntervals

from pytorch_lightning import Trainer
trainer = Trainer(
    max_steps=4,
    logger=False,
    enable_progress_bar=False,
    enable_model_summary=False
)

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="optuna")


# ==================================================
# REPRODUCTIBILITY
# ==================================================
import random
import torch

SEED = 1
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

2026-03-01 20:09:02,312	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2026-03-01 20:09:02,740	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
💡 Tip: For seamless cloud logging and experiment tracking, try installing [litlogger](https://pypi.org/project/litlogger/) to enable LitLogger, which logs metrics and artifacts automatically to the Lightning Experiments platform.


### Results Storage

In [4]:
from pathlib import Path

BASE_RESULTS = Path("Results/RQ3")
BASE_RESULTS.mkdir(parents=True, exist_ok=True)

def save_results(df, model_family, pollutant, horizon_label, fold=None):

    out_dir = BASE_RESULTS / model_family
    out_dir.mkdir(parents=True, exist_ok=True)

    if fold is not None:
        fname = f"{pollutant}_{fold}_{horizon_label}.csv"
    else:
        fname = f"{pollutant}_{horizon_label}.csv"

    df.to_csv(out_dir / fname, index=False)

## Data

In [5]:
# ===============================
# DATA
# ===============================
df = pd.read_parquet(r'..\Data\CAMS\processed\eac4_era5_2010_2024_brasil_enhanced.parquet')


# ===============================
# POLLUTANTS
# ===============================
pm2p5 = (
    df
    .copy()
    .rename(columns={
        'pm2p5': 'y',
        'valid_time': 'ds'        
    })
)

go3 = (
    df
    .copy()
    .rename(columns={
        'go3': 'y',
        'valid_time': 'ds'        
    })
)

In [6]:
BASE_COLS = [ "ds", "unique_id", "y"]

O3_F1 = [ "no2", "co"]
O3_F2 = [ "u10", "v10", "wind_speed", "t2m", "d2m", "RH", "ssrd_flux", "blh"]
O3_F3 = O3_F1 + O3_F2

PM_F1 = [ "so2", "no2", "co"]
PM_F2 = ["u10", "v10", "wind_speed", "t2m", "d2m", "RH", "ssrd_flux", "blh"]
PM_F3 = PM_F1 + PM_F2

In [7]:
pollutant_dict = {
    'go3': {
        'f0': {
            'df': go3[BASE_COLS].copy(),
            'exog_cols': []
        },
        'f1': {
            'df': go3[BASE_COLS + O3_F1].copy(),
            'exog_cols': O3_F1
        },
        'f2': {
            'df': go3[BASE_COLS + O3_F2].copy(),
            'exog_cols': O3_F2
        },
        'f3': {
            'df': go3[BASE_COLS + O3_F3].copy(),
            'exog_cols': O3_F3
        },
        'scaler': 1e8
    },

    'pm2p5': {
        'f0': {
            'df': pm2p5[BASE_COLS].copy(),
            'exog_cols': []
        },
        'f1': {
            'df': pm2p5[BASE_COLS + PM_F1].copy(),
            'exog_cols': PM_F1
        },
        'f2': {
            'df': pm2p5[BASE_COLS + PM_F2].copy(),
            'exog_cols': PM_F2
        },
        'f3': {
            'df': pm2p5[BASE_COLS + PM_F3].copy(),
            'exog_cols': PM_F3
        },
        'scaler': 1e9
    }
}

### Setup

In [8]:
# ===============================
# CONFIG
# ===============================
steps_per_day = 8
two_year_steps = 2 * 365 * steps_per_day
target_windows = 30

FREQ = '3h'
SEASON_LENGTH = 8 

experiments_dict = {
    # '1 days': {
    #     'horizon': 8*1,
    #     'step_size': max(8*1, two_year_steps // target_windows),
    #     'windows': target_windows,
    # },
    '7 days': {
        'horizon': 8*7,
        'step_size': max(8*7, two_year_steps // target_windows),
        'windows': target_windows,
    },
    '14 days': {
        'horizon': 8*14,
        'step_size': max(8*14, two_year_steps // target_windows),
        'windows': target_windows,
    },
    '30 days': {
        'horizon': 8*30,
        'step_size': 8*30,
        'windows': two_year_steps // (8*30),  # 24
    },
}

# **Models**

### Neural

In [ ]:
for pollutant_name, pollutant_ in pollutant_dict.items():
    
    scaler = pollutant_['scaler']
    
    for fk_label in ['f0', 'f1', 'f2', 'f3']:
        
        for horizon_label, experiment_ in experiments_dict.items():
            
            try:
                clear_output(wait=True)
                print(f"Running RQ3 | {pollutant_name} | {fk_label} | {horizon_label}")

                # --------------------------------
                # DATA (copy fresh every iteration)
                # --------------------------------
                df_config = pollutant_[fk_label]['df'].copy()
                exog_cols = pollutant_[fk_label]['exog_cols']

                # Scale target
                df_config['y'] = df_config['y'] * scaler

                # --------------------------------
                # MODELS
                # --------------------------------
                models = [
                    GRU(
                        h=experiment_['horizon'],
                        input_size=14*8,
                        max_steps=1000,
                        learning_rate=1e-3,
                        batch_size=32,
                        windows_batch_size=1024,
                        random_seed=SEED,
                        alias='GRU',
                        loss=MAE(),
                        hist_exog_list=exog_cols if exog_cols else None,
                        logger=False,
                    ),
                    LSTM(
                        h=experiment_['horizon'],
                        input_size=14*8,
                        max_steps=1000,
                        learning_rate=1e-3,
                        batch_size=32,
                        windows_batch_size=1024,
                        random_seed=SEED,
                        alias='LSTM',
                        loss=MAE(),
                        hist_exog_list=exog_cols if exog_cols else None,
                        logger=False,
                    ),
                    NBEATSx(
                        h=experiment_['horizon'],
                        input_size=14*8,
                        stack_types=["identity", "trend", "seasonality"],
                        n_blocks=[1, 1, 1],
                        mlp_units=3 * [[256, 256]],
                        shared_weights=True,
                        activation='ReLU',
                        max_steps=1000,
                        learning_rate=1e-3,
                        batch_size=32,
                        windows_batch_size=1024,
                        random_seed=SEED,
                        alias='NBEATSx-I',
                        hist_exog_list=exog_cols if exog_cols else None,
                        loss=MAE(),
                    ),
                    NBEATSx(
                        h=experiment_['horizon'],
                        input_size=14*8,
                        stack_types=['identity'] * 3,
                        n_blocks=[2, 2, 2],
                        mlp_units=3 * [[256, 256]],
                        shared_weights=False,
                        activation='ReLU',
                        max_steps=1000,
                        learning_rate=1e-3,
                        batch_size=32,
                        windows_batch_size=1024,
                        random_seed=SEED,
                        alias='NBEATSx-G',
                        logger=False,
                        hist_exog_list=exog_cols if exog_cols else None,
                        loss=MAE(),
                    ),
                    NHITS(
                        h=experiment_['horizon'],
                        input_size=14*8,
                        n_blocks=[1, 1, 1],
                        mlp_units=3 * [[256, 256]],
                        n_pool_kernel_size=[2, 2, 1],
                        n_freq_downsample=[4, 2, 1],
                        activation='ReLU',
                        dropout_prob_theta=0.1,
                        max_steps=1000,
                        learning_rate=1e-3,
                        batch_size=32,
                        windows_batch_size=1024,
                        random_seed=SEED,
                        alias='NHITS',
                        logger=False,
                        hist_exog_list=exog_cols if exog_cols else None,
                        loss=MAE(),
                    ),
                ]

                nf = NeuralForecast(models=models, freq=FREQ)

                # --------------------------------
                # CROSS VALIDATION
                # --------------------------------
                start_time = time.time()

                results_nf = nf.cross_validation(
                    df=df_config,
                    h=experiment_['horizon'],
                    n_windows=experiment_['windows'],
                    step_size=experiment_['step_size'],
                    refit=False,
                )

                end_time = time.time()
                total_time = end_time - start_time

                results_nf['fit_time_seconds'] = total_time

                # --------------------------------
                # SCALE BACK
                # --------------------------------
                for col in ['y', 'GRU', 'LSTM', 'NBEATSx-I', 'NBEATSx-G', 'NHITS']:
                    if col in results_nf.columns:
                        results_nf[col] = results_nf[col] / scaler

                # --------------------------------
                # SAVE
                # --------------------------------
                save_results(
                    results_nf,
                    model_family="dl",
                    pollutant=pollutant_name,
                    horizon_label=horizon_label.replace(" ", ""),
                    fold=fk_label
                )

                # --------------------------------
                # CLEAN MEMORY
                # --------------------------------
                del nf
                del models
                gc.collect()
                torch.cuda.empty_cache()

            except Exception as e:
                print(f"Failed on {pollutant_name} {fk_label} {horizon_label}: {e}")
                continue

### Machine Learners

In [ ]:
for pollutant_name, pollutant_ in pollutant_dict.items():
    
    scaler = pollutant_['scaler']
    
    for fk_label in ['f0', 'f1', 'f2', 'f3']:
        
        for horizon_label, experiment_ in experiments_dict.items():
            
            try:
                clear_output(wait=True)
                print(f"Running RQ3-ML | {pollutant_name} | {fk_label} | {horizon_label}")

                # --------------------------------
                # DATA (copy fresh every iteration)
                # --------------------------------
                df_config = pollutant_[fk_label]['df']
                df_config = df_config.sort_values(['unique_id', 'ds'])
                
                # --------------------------------
                # MODEL (LightGBM only)
                # --------------------------------
                lgb_model = lgb.LGBMRegressor(
                    n_estimators=500,
                    learning_rate=0.05,
                    num_leaves=31,
                    subsample=0.8,
                    colsample_bytree=0.8,
                    random_state=SEED,
                    n_jobs=10,
                )

                # --------------------------------
                # MLForecast
                # --------------------------------
                mlf = MLForecast(
                    models={'LightGBM': lgb_model},
                    freq=FREQ,
                    lags=[1, 2, 4, 8, 16, 24, 56, 112],
                    lag_transforms={
                        8:  [RollingMean(window_size=8), ExpandingMean()],
                        56: [RollingMean(window_size=56)],
                    },
                    date_features=[
                        'hour',
                        'dayofweek',
                        'month',
                        'dayofyear',
                    ],
                )

                # --------------------------------
                # CROSS VALIDATION
                # --------------------------------
                start_time = time.time()

                results_ml = mlf.cross_validation(
                    df=df_config,
                    h=experiment_['horizon'],
                    n_windows=experiment_['windows'],
                    step_size=experiment_['step_size'],
                    refit=False,
                )

                end_time = time.time()
                total_time = end_time - start_time

                results_ml['fit_time_seconds'] = total_time

                # --------------------------------
                # SAVE
                # --------------------------------
                save_results(
                    results_ml,
                    model_family="ml",
                    pollutant=pollutant_name,
                    horizon_label=horizon_label.replace(" ", ""),
                    fold=fk_label
                )

                # --------------------------------
                # CLEAN MEMORY
                # --------------------------------
                del mlf
                del lgb_model
                gc.collect()

            except Exception as e:
                print(f"Failed on {pollutant_name} {fk_label} {horizon_label}: {e}")
                continue

Running RQ3-ML | pm2p5 | f3 | 30 days
Failed on pm2p5 f3 30 days: Unable to allocate 96.3 MiB for an array with shape (1, 12623616) and data type int64


## Merge

In [50]:
BASE_RESULTS = Path("Results/RQ2")
FULL_DIR = BASE_RESULTS / "full"
FULL_DIR.mkdir(parents=True, exist_ok=True)

families = ["ml", "dl", "neural"]

def build_full_results(pollutant, fold, horizon_label):
    """
    Concatena famílias horizontalmente via concat,
    garantindo que não haja perda de linhas.
    """

    dfs = []

    for family in families:
        fpath = BASE_RESULTS / family / f"{pollutant}_{fold}_{horizon_label.replace(' ', '')}.csv"

        if fpath.exists():
            df = pd.read_csv(fpath)

            if "fit_time_seconds" in df.columns:
                df = df.drop(columns=["fit_time_seconds"])

            dfs.append(df)
        else:
            print(f"⚠️ Missing: {fpath}")

    if not dfs:
        return None

    # Ordenar todos igualmente
    for i in range(len(dfs)):
        dfs[i] = dfs[i].sort_values(
            ["unique_id", "ds", "cutoff", "fold"]
        ).reset_index(drop=True)

    # Usar o primeiro como base
    base = dfs[0][["unique_id", "ds", "cutoff", "y", "fold"]].copy()

    # Adicionar modelos das outras famílias
    for df in dfs:

        model_cols = [
            c for c in df.columns
            if c not in ["unique_id", "ds", "cutoff", "y", "fold"]
        ]

        base = pd.concat(
            [base, df[model_cols]],
            axis=1
        )

    return base


for pollutant_name, pollutant_dict in pollutant_dict.items():

    for fold_name in pollutant_dict.keys():

        if fold_name == "scaler":
            continue

        for horizon_label in experiments_dict.keys():

            horizon_clean = horizon_label.replace(" ", "")

            print(f"Building FULL | {pollutant_name} | {fold_name} | {horizon_clean}")

            df_full = build_full_results(
                pollutant=pollutant_name,
                fold=fold_name,
                horizon_label=horizon_clean
            )

            if df_full is None:
                continue

            df_full.to_csv(
                FULL_DIR / f"{pollutant_name}_{fold_name}_{horizon_clean}.csv",
                index=False
            )

Building FULL | go3 | sudeste | 7days
Building FULL | go3 | sudeste | 30days
Building FULL | go3 | sul | 7days
Building FULL | go3 | sul | 30days
Building FULL | go3 | centro_oeste | 7days
Building FULL | go3 | centro_oeste | 30days
Building FULL | go3 | coastal | 7days
Building FULL | go3 | coastal | 30days
Building FULL | pm2p5 | sudeste | 7days
Building FULL | pm2p5 | sudeste | 30days
Building FULL | pm2p5 | sul | 7days
Building FULL | pm2p5 | sul | 30days
Building FULL | pm2p5 | centro_oeste | 7days
Building FULL | pm2p5 | centro_oeste | 30days
Building FULL | pm2p5 | coastal | 7days
Building FULL | pm2p5 | coastal | 30days


## **Estatísticas**

In [ ]:
# ==================================================
# METRICS
# ==================================================

def mae(y, yhat):
    return np.mean(np.abs(y - yhat))

def mse(y, yhat):
    return np.mean((y - yhat) ** 2)

def rmse(y, yhat):
    return np.sqrt(mse(y, yhat))

def smape(y, yhat):
    denom = (np.abs(y) + np.abs(yhat)) / 2
    mask = denom != 0
    if mask.sum() == 0:
        return np.nan
    return np.mean(np.abs(y[mask] - yhat[mask]) / denom[mask])

def mae_conditional(y, yhat, threshold):
    mask = y >= threshold
    if mask.sum() == 0:
        return np.nan
    return np.mean(np.abs(y[mask] - yhat[mask]))

def bias_conditional(y, yhat, threshold):
    mask = y >= threshold
    if mask.sum() == 0:
        return np.nan
    return np.mean(yhat[mask] - y[mask])

def skill_score(model_err, baseline_err):
    if baseline_err == 0 or np.isnan(baseline_err):
        return np.nan
    return 1 - model_err / baseline_err

def extreme_event_metrics(y, yhat, threshold):
    y_event = y >= threshold
    yhat_event = yhat >= threshold

    tp = np.sum(y_event & yhat_event)
    fp = np.sum(~y_event & yhat_event)
    fn = np.sum(y_event & ~yhat_event)

    precision = tp / (tp + fp) if (tp + fp) > 0 else np.nan
    recall = tp / (tp + fn) if (tp + fn) > 0 else np.nan

    if precision > 0 and recall > 0:
        f1 = 2 * precision * recall / (precision + recall)
    else:
        f1 = np.nan

    return precision, recall, f1


def mase_denom(y_train, m=1):
    """
    Mean absolute error of seasonal naive (lag m) on in-sample training.
    This is the standard denominator of MASE.
    """
    y_train = np.asarray(y_train)
    if len(y_train) <= m:
        return np.nan
    return np.mean(np.abs(y_train[m:] - y_train[:-m]))

def mase(y_true, y_pred, y_train, m=1):
    """
    MASE = MAE(test) / MAE(seasonal naive in-sample, lag m)
    """
    denom = mase_denom(y_train, m=m)
    if denom == 0 or np.isnan(denom):
        return np.nan
    return mae(y_true, y_pred) / denom


# ==================================================
# CONFIG
# ==================================================

RQ2_BASE = Path("Results/RQ2/full")
RQ1_BASE = Path("Results/RQ1/full")

baseline_name = "Naive"
WINDOW_P95 = 365 * 8  # 1 ano em 3h

MASE_M = 56  # weekly seasonality for 3-hourly data

records = []


# ==================================================
# LOOP RQ2 FILES
# ==================================================

for file in RQ2_BASE.glob("*.csv"):

    # Ex: go3_sudeste_1days.csv
    parts = file.stem.split("_")

    pollutant = parts[0]
    horizon = parts[-1]
    fold = "_".join(parts[1:-1])

    print(f"Evaluating | {pollutant} | {fold} | {horizon}")

    df_pred = pd.read_csv(file)
    df_pred["ds"] = pd.to_datetime(df_pred["ds"])
    df_pred["cutoff"] = pd.to_datetime(df_pred["cutoff"])

    # --------------------------------------
    # GROUND TRUTH (VINDO DO pollutant_dict)
    # --------------------------------------
    test_df = pollutant_dict[pollutant][fold]["test_df"].copy()
    test_df["ds"] = pd.to_datetime(test_df["ds"])

    # --------------------------------------
    # LOAD RQ1 (PODE NÃO EXISTIR PARA ALGUNS IDs)
    # --------------------------------------
    rq1_path = RQ1_BASE / f"{pollutant}_{horizon}.csv"

    if rq1_path.exists():
        df_rq1 = pd.read_csv(rq1_path)
        df_rq1["ds"] = pd.to_datetime(df_rq1["ds"])
        df_rq1["cutoff"] = pd.to_datetime(df_rq1["cutoff"])

        df_pred = df_pred.merge(
            df_rq1,
            on=["unique_id", "ds", "cutoff"],
            suffixes=("", "_RQ1"),
            how="left"
        )

    # --------------------------------------
    # MODELOS
    # --------------------------------------
    model_cols = [
        c for c in df_pred.columns
        if (
            c not in ["unique_id", "ds", "cutoff", "y", "fold"]
            and not c.endswith("_RQ1")
            and df_pred[c].dtype in [np.float64, np.float32]
        )
    ]

    # --------------------------------------
    # LOOP POR UNIQUE_ID + CUTOFF
    # --------------------------------------
    for (uid, cutoff), df_fold in df_pred.groupby(["unique_id", "cutoff"]):

        y_train_full = (
            test_df
            .query("unique_id == @uid and ds <= @cutoff")["y"]
            .values
        )

        if len(y_train_full) < WINDOW_P95:
            continue

        p95 = np.percentile(y_train_full[-WINDOW_P95:], 95)

        # ---- MASE denominator (computed once per uid+cutoff)
        mase_denom_uid = mase_denom(y_train_full, m=MASE_M)

        y_full = df_fold["y"].values
        y_base_full = df_fold[baseline_name].values

        for model in model_cols:

            yhat_full = df_fold[model].values

            mask_valid = ~np.isnan(yhat_full)
            if mask_valid.sum() == 0:
                continue

            y_valid = y_full[mask_valid]
            yhat = yhat_full[mask_valid]
            y_base_valid = y_base_full[mask_valid]

            # ================= RQ2 =================
            mae_model = mae(y_valid, yhat)
            rmse_model = rmse(y_valid, yhat)
            smape_model = smape(y_valid, yhat)
            mae_p95 = mae_conditional(y_valid, yhat, p95)

            mae_base = mae(y_valid, y_base_valid)
            rmse_base = rmse(y_valid, y_base_valid)
            smape_base = smape(y_valid, y_base_valid)
            mae_base_p95 = mae_conditional(y_valid, y_base_valid, p95)

            skill_mae = skill_score(mae_model, mae_base)
            skill_rmse = skill_score(rmse_model, rmse_base)
            skill_smape = skill_score(smape_model, smape_base)
            skill_p95 = skill_score(mae_p95, mae_base_p95)

            # ---- MASE + Skill(MASE) vs baseline
            mase_model = np.nan if (np.isnan(mase_denom_uid) or mase_denom_uid == 0) else mae_model / mase_denom_uid
            mase_base = np.nan if (np.isnan(mase_denom_uid) or mase_denom_uid == 0) else mae_base / mase_denom_uid
            skill_mase = skill_score(mase_model, mase_base)

            bias_p95 = bias_conditional(y_valid, yhat, p95)
            precision, recall, f1 = extreme_event_metrics(y_valid, yhat, p95)

            # ================= RQ1 =================
            model_rq1_col = model + "_RQ1"

            if model_rq1_col in df_fold.columns:
                yhat_rq1 = df_fold[model_rq1_col].values
                mask_rq1 = ~np.isnan(yhat_rq1)
                mask_joint = mask_valid & mask_rq1

                if mask_joint.sum() > 0:
                    y_joint = y_full[mask_joint]
                    yhat_rq1_joint = yhat_rq1[mask_joint]

                    mae_rq1 = mae(y_joint, yhat_rq1_joint)
                    rmse_rq1 = rmse(y_joint, yhat_rq1_joint)
                    smape_rq1 = smape(y_joint, yhat_rq1_joint)
                    mae_p95_rq1 = mae_conditional(y_joint, yhat_rq1_joint, p95)

                    skill_mae_vs_rq1 = skill_score(mae_model, mae_rq1)
                    skill_rmse_vs_rq1 = skill_score(rmse_model, rmse_rq1)
                    skill_smape_vs_rq1 = skill_score(smape_model, smape_rq1)
                    skill_p95_vs_rq1 = skill_score(mae_p95, mae_p95_rq1)

                    # ---- MASE vs RQ1 (same denominator, since y_train_full is fixed)
                    mase_rq1 = np.nan if (np.isnan(mase_denom_uid) or mase_denom_uid == 0) else mae_rq1 / mase_denom_uid
                    skill_mase_vs_rq1 = skill_score(mase_model, mase_rq1)
                else:
                    mae_rq1 = rmse_rq1 = smape_rq1 = mae_p95_rq1 = np.nan
                    skill_mae_vs_rq1 = skill_rmse_vs_rq1 = skill_smape_vs_rq1 = skill_p95_vs_rq1 = np.nan
                    mase_rq1 = skill_mase_vs_rq1 = np.nan
            else:
                mae_rq1 = rmse_rq1 = smape_rq1 = mae_p95_rq1 = np.nan
                skill_mae_vs_rq1 = skill_rmse_vs_rq1 = skill_smape_vs_rq1 = skill_p95_vs_rq1 = np.nan
                mase_rq1 = skill_mase_vs_rq1 = np.nan

            # ================= SAVE =================
            records.append({
                "pollutant": pollutant,
                "fold": fold,
                "horizon": horizon,
                "unique_id": uid,
                "cutoff": cutoff,
                "model": model,

                "MAE": mae_model,
                "RMSE": rmse_model,
                "sMAPE": smape_model,
                "MAE_p95": mae_p95,

                "Skill_MAE": skill_mae,
                "Skill_RMSE": skill_rmse,
                "Skill_sMAPE": skill_smape,
                "Skill_p95": skill_p95,

                "MASE": mase_model,
                "MASE_base": mase_base,
                "Skill_MASE": skill_mase,
                "MASE_denom": mase_denom_uid,
                "MASE_m": MASE_M,

                "Bias_p95": bias_p95,
                "Precision_p95": precision,
                "Recall_p95": recall,
                "F1_p95": f1,

                "MAE_RQ1": mae_rq1,
                "RMSE_RQ1": rmse_rq1,
                "sMAPE_RQ1": smape_rq1,
                "MAE_p95_RQ1": mae_p95_rq1,

                "Skill_MAE_vs_RQ1": skill_mae_vs_rq1,
                "Skill_RMSE_vs_RQ1": skill_rmse_vs_rq1,
                "Skill_sMAPE_vs_RQ1": skill_smape_vs_rq1,
                "Skill_p95_vs_RQ1": skill_p95_vs_rq1,

                "MASE_RQ1": mase_rq1,
                "Skill_MASE_vs_RQ1": skill_mase_vs_rq1,
            })


# ==================================================
# SAVE
# ==================================================

metrics_df = pd.DataFrame(records)
metrics_df.to_csv("Results/RQ2/metrics.csv", index=False)

print("Done.")